In [15]:
import requests
import pandas as pd

API_KEY = "NGELwQY8b2XQcUu8sAs9a96T9COtSsacKzI0FK4gJmS0qgqGjTRWgRBfNG3sfwt0"
BASE_URL = "https://api.collegefootballdata.com/drives"

years = [2024,2023,2022]
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

params_list = [{'year': year, 'week': week, 'classification': 'fbs'} for year in years for week in weeks]

headers = {'Authorization': f'Bearer {API_KEY}'}

def get_data_from_api(params):
    res = requests.get(BASE_URL, params=params, headers=headers)
    if res.status_code == 200:
        return res.json()
    else:
        print(f"Error fetching data for year {params['year']} week {params['week']}: {res.status_code}")
        return None

data = []
for params in params_list:
    week_data = get_data_from_api(params)
    if week_data:
        for play in week_data:
            play['week'] = params['week']
            play['year'] = params['year']
        data.extend(week_data)

df = pd.DataFrame(data)

print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62387 entries, 0 to 62386
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   offense              62387 non-null  object
 1   offense_conference   62387 non-null  object
 2   defense              62387 non-null  object
 3   defense_conference   62387 non-null  object
 4   game_id              62387 non-null  int64 
 5   id                   62387 non-null  object
 6   drive_number         62387 non-null  int64 
 7   scoring              62387 non-null  bool  
 8   start_period         62387 non-null  int64 
 9   start_yardline       62387 non-null  int64 
 10  start_yards_to_goal  62387 non-null  int64 
 11  start_time           62387 non-null  object
 12  end_period           62387 non-null  int64 
 13  end_yardline         62387 non-null  int64 
 14  end_yards_to_goal    62387 non-null  int64 
 15  end_time             62387 non-null  object
 16  elap

In [21]:
df["distance_to_goal"] = 100 - df["start_yards_to_goal"]

df["points_scored_offense"] = df["end_offense_score"] - df["start_offense_score"]
df["points_scored_defense"] = df["end_defense_score"] - df["start_defense_score"]

df['score_diff_start'] = df['start_offense_score'] - df['start_defense_score']
df['yards_gained'] = df['end_yardline'] - df['start_yardline']

df.loc[(df["end_yardline"] < 0) & (df["drive_result"] == "Uncategorized"), "end_yardline"] = 0
df.loc[(df["end_yards_to_goal"] < 0) & (df["drive_result"] == "Uncategorized"), "end_yards_to_goal"] = 0

uncategorized_issues = df[(df["end_yards_to_goal"] < 0) & (df["drive_result"] == "Uncategorized")]
df = df[~(df['drive_result'] == 'Uncategorized')]

print(uncategorized_issues)
print(len(df['offense'].unique()))
print(df.info())

Empty DataFrame
Columns: [offense, offense_conference, defense, defense_conference, game_id, id, drive_number, scoring, start_period, start_yardline, start_yards_to_goal, start_time, end_period, end_yardline, end_yards_to_goal, end_time, elapsed, plays, yards, drive_result, is_home_offense, start_offense_score, start_defense_score, end_offense_score, end_defense_score, week, year, distance_to_goal, points_scored_offense, points_scored_defense, yards_gained_per_play, score_diff_start, yards_gained, start_time_minutes, start_time_seconds, end_time_minutes, end_time_seconds, start_time_in_seconds, end_time_in_seconds, offense_defense_matchup_encoded, offense_idx, defense_idx, offense_conference_idx, defense_conference_idx, offense_defense_matchup, offense_defense_matchup_avg_points_by_offense_points, offense_defense_matchup_avg_points_by_defense_points, matchup_avg_yards, goal_to_go, is_off_adv, is_def_adv, score_diff_start_off_adv, score_diff_start_def_adv]
Index: []

[0 rows x 53 column

In [22]:
import ast

def extract_time(time_str):
    try:
        time_dict = ast.literal_eval(time_str) 
        minutes = time_dict.get("minutes", 0)
        seconds = time_dict.get("seconds", 0)
        return minutes, seconds
    except:
        return 0, 0

df[['start_time_minutes', 'start_time_seconds']] = df['start_time'].apply(lambda x: pd.Series(extract_time(x)))
df[['end_time_minutes', 'end_time_seconds']] = df['end_time'].apply(lambda x: pd.Series(extract_time(x)))

df['start_time_in_seconds'] = df['start_time_minutes']*60 + df['start_time_seconds']
df['end_time_in_seconds'] = df['end_time_minutes']*60 + df['end_time_seconds']

df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 62076 entries, 0 to 62386
Data columns (total 53 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   offense                                               62076 non-null  object 
 1   offense_conference                                    62076 non-null  object 
 2   defense                                               62076 non-null  object 
 3   defense_conference                                    62076 non-null  object 
 4   game_id                                               62076 non-null  int64  
 5   id                                                    62076 non-null  object 
 6   drive_number                                          62076 non-null  int64  
 7   scoring                                               62076 non-null  bool   
 8   start_period                                          62076 n

(None,
             offense offense_conference           defense defense_conference  \
 0  Western Kentucky     Conference USA           Alabama                SEC   
 1           Alabama                SEC  Western Kentucky     Conference USA   
 2  Western Kentucky     Conference USA           Alabama                SEC   
 3           Alabama                SEC  Western Kentucky     Conference USA   
 4  Western Kentucky     Conference USA           Alabama                SEC   
 
      game_id          id  drive_number  scoring  start_period  start_yardline  \
 0  401628319  4016283191             1    False             1              75   
 1  401628319  4016283192             2    False             1              31   
 2  401628319  4016283193             3    False             1              80   
 3  401628319  4016283194             4     True             1              84   
 4  401628319  4016283195             5    False             1              75   
 
    ...  defense_

In [23]:
from sklearn.preprocessing import LabelEncoder

# Label encoding offense-defense matchups
matchup_encoder = LabelEncoder()
df["offense_defense_matchup_encoded"] = matchup_encoder.fit_transform(df["offense"] + "_" + df["defense"])
df['offense_idx'] = matchup_encoder.fit_transform(df['offense'])
df['defense_idx'] = matchup_encoder.fit_transform(df['defense'])
df['offense_conference_idx'] = matchup_encoder.fit_transform(df['offense_conference'])
df['defense_conference_idx'] = matchup_encoder.fit_transform(df['defense_conference'])
# Compute average points scored per matchup
df["offense_defense_matchup"] = df["offense"] + "_" + df["defense"]
matchup_avg_points_by_offense = df.groupby("offense_defense_matchup")["points_scored_offense"].mean()
matchup_avg_points_by_defense = df.groupby("offense_defense_matchup")["points_scored_defense"].mean()
df["offense_defense_matchup_avg_points_by_offense_points"] = df["offense_defense_matchup"].map(matchup_avg_points_by_offense)
df["offense_defense_matchup_avg_points_by_defense_points"] = df["offense_defense_matchup"].map(matchup_avg_points_by_defense)
matchup_avg_yards = df.groupby('offense_defense_matchup')['yards_gained'].mean()
df['matchup_avg_yards'] = df['offense_defense_matchup'].map(matchup_avg_yards)
df['is_home_offense'] = df['is_home_offense'].astype(int)
df.info()

df

<class 'pandas.core.frame.DataFrame'>
Index: 62076 entries, 0 to 62386
Data columns (total 53 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   offense                                               62076 non-null  object 
 1   offense_conference                                    62076 non-null  object 
 2   defense                                               62076 non-null  object 
 3   defense_conference                                    62076 non-null  object 
 4   game_id                                               62076 non-null  int64  
 5   id                                                    62076 non-null  object 
 6   drive_number                                          62076 non-null  int64  
 7   scoring                                               62076 non-null  bool   
 8   start_period                                          62076 n

,offense,offense_conference,defense,defense_conference,game_id,id,drive_number,scoring,start_period,start_yardline,...,defense_conference_idx,offense_defense_matchup,offense_defense_matchup_avg_points_by_offense_points,offense_defense_matchup_avg_points_by_defense_points,matchup_avg_yards,goal_to_go,is_off_adv,is_def_adv,score_diff_start_off_adv,score_diff_start_def_adv
0,Western Kentucky,Conference USA,Alabama,SEC,401628319,4016283191,1,False,1,75,...,21,Western Kentucky_Alabama,0.00,0.000000,-10.428571,0,0,0,0,0
1,Alabama,SEC,Western Kentucky,Conference USA,401628319,4016283192,2,False,1,31,...,9,Alabama_Western Kentucky,4.50,0.000000,43.714286,0,0,0,0,0
2,Western Kentucky,Conference USA,Alabama,SEC,401628319,4016283193,3,False,1,80,...,21,Western Kentucky_Alabama,0.00,0.000000,-10.428571,0,0,0,0,0
3,Alabama,SEC,Western Kentucky,Conference USA,401628319,4016283194,4,True,1,84,...,9,Alabama_Western Kentucky,4.50,0.000000,43.714286,0,0,0,0,0
4,Western Kentucky,Conference USA,Alabama,SEC,401628319,4016283195,5,False,1,75,...,21,Western Kentucky_Alabama,0.00,0.000000,-10.428571,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62381,Navy,American Athletic,Army,FBS Independents,401404145,40140414524,24,False,4,75,...,10,Navy_Army,0.36,0.640000,-21.000000,0,0,0,0,0
62382,Army,FBS Independents,Navy,American Athletic,401404145,40140414525,25,False,4,7,...,1,Army_Navy,0.25,0.416667,-7.375000,0,0,0,0,0
62383,Army,FBS Independents,Navy,American Athletic,401404145,40140414526,26,True,5,75,...,1,Army_Navy,0.25,0.416667,-7.375000,0,1,0,1,0
62384,Navy,American Athletic,Army,FBS Independents,401404145,40140414527,27,True,5,97,...,10,Navy_Army,0.36,0.640000,-21.000000,0,0,0,0,0


In [24]:
df['goal_to_go'] = (df['start_yards_to_goal'] <= 10).astype(int)
df['is_off_adv'] = (df['start_offense_score'] > df['start_defense_score']).astype(int)
df['is_def_adv'] = (df['start_defense_score'] > df['start_offense_score']).astype(int)

df['score_diff_start_off_adv'] = ((df['start_offense_score'] - df['start_defense_score'])>0).astype(int)
df['score_diff_start_def_adv'] = ((df['start_defense_score'] - df['start_offense_score'])>0).astype(int)
df.to_csv('drives.csv', index=False)